In [1]:
# bash 셸로 명령어 입력하여 라이브러리 설치하기

%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
pip install gensim
pip install tqdm

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [2]:
# JAVA_HOME 환경변수 설정하기

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [4]:
raw_file = open('/content/drive/MyDrive/Project/NLPModel/Data/WordDict.txt', 'r')
train_data = raw_file.readlines()
raw_file.close()
regx_data = []

for s in train_data:
  regx_data.append(s.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]",""))

print(regx_data[100])

다음 주에 할아버지 생신이래요



In [5]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

sf = open('/content/drive/MyDrive/Project/NLPModel/Data/stopword.txt', 'r')

stopwords = sf.readlines()

tokenized_data = []
for sentence in tqdm(regx_data):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 22010/22010 [00:28<00:00, 782.37it/s]


In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 1, window = 5, min_count = 5, workers = 4, sg = 0)

model.save('/content/drive/MyDrive/Project/NLPModel/EmbeddingModel2.model')

In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('/content/drive/MyDrive/Project/NLPModel/EmbeddingModel2.model')
temp = []
temp.append(model.wv['아빠'])
print(temp[0])


[-1.3943958]


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from konlpy.tag import Okt
import csv
import numpy as np

train_data = pd.read_csv('/content/drive/MyDrive/Project/NLPModel/Data/TrainData.csv')
train_data = train_data.to_numpy()

tokenized_data = []

sf = open('/content/drive/MyDrive/Project/NLPModel/Data/stopword.txt', 'r')

stopwords = sf.readlines()

max = 0
okt = Okt()

for line in tqdm(train_data):
  temp = []

  sentence = line[2].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
  tokenized_sentence = okt.morphs(sentence, stem=True)
  stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]

  if max < len(stopwords_removed_sentence):
    max = len(stopwords_removed_sentence)
  
  temp.append(stopwords_removed_sentence)
  temp.append(line[3])

  tokenized_data.append(temp)




embedding_data = []
NE = 0
model = Word2Vec.load('/content/drive/MyDrive/Project/NLPModel/EmbeddingModel2.model')
print(max)

for line in tqdm(tokenized_data):
  temp = []
  for word in line[0]:
    if word not in model:
      temp.append(NE)
    else:
      temp.append(model[word].tolist())

  if(len(temp) < max):
    for i in range(max - len(temp)):
      temp.append(NE)

  temp.append(line[1])

  embedding_data.append(temp)


df2 = pd.DataFrame(embedding_data)
df2.to_csv('/content/drive/MyDrive/Project/NLPModel/Data/EmbeddingTrainData2.csv')

100%|██████████| 22589/22589 [00:24<00:00, 920.69it/s]


28


  0%|          | 0/22589 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
 42%|████▏     | 9526/22589 [00:00<00:00, 18353.92it/s]